In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models


In [ ]:
train_data = keras.preprocessing.image_dataset_from_directory(
    "dataset/train",
    image_size=(224, 224),
    batch_size=32
)

val_data = keras.preprocessing.image_dataset_from_directory(
    "dataset/val",
    image_size=(224, 224),
    batch_size=32
)


In [ ]:
normalization_layer = layers.Rescaling(1./255)

train_data = train_data.map(lambda x, y: (normalization_layer(x), y))
val_data = val_data.map(lambda x, y: (normalization_layer(x), y))


### Do NOT apply augmentation to validation/test data.

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])


In [ ]:
train_data = train_data.map(
    lambda x, y: (data_augmentation(x, training=True), y)
)


In [ ]:
base_model = keras.applications.ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)


In [ ]:
base_model.trainable = False


In [ ]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")  # binary classification
])


In [ ]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
history = model.fit(
    train_data,
    epochs=5,
    validation_data=val_data
)


In [ ]:
model.evaluate(val_data)


In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False


In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
model.fit(
    train_data,
    epochs=5,
    validation_data=val_data
)


In [ ]:
model.evaluate(val_data)


In [ ]:
predictions = model.predict(val_data)


In [ ]:
model.save("cnn_with_augmentation.h5")


### Load data → Normalize → Augment → Pretrained CNN → Freeze → Train → Fine-tune → Evaluate → Save
